# Supervised Learning Problem

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.in2p3.fr%2Fenergy4climate%2Fpublic%2Feducation%2Fmachine_learning_for_climate_and_energy/master?filepath=book%2Fnotebooks%2F02_supervised_learning_problem.ipynb)

<div class="alert alert-block alert-warning">
    <b>Prerequisites</b>
    
- [Elements of Probability Theory](appendix_elements_of_probability_theory.ipynb)  
- Define *supervised* and *unsupervised* learning
- Give the difference between *qualitative* and *quantitative* variables and define of *regression* and *classification*
</div>

<div class="alert alert-block alert-info">
    <b>Learning Outcomes</b>
    
- Define a supervised learning problem
</div>

## Example: Home Heating

<img alt="Thermostat" src="images/erik-mclean-fSLI8RdCdyk-unsplash.jpg" width=500  style="float:right">

When its cold in my accommodation, I heat it.

$\rightarrow$ I suspect a relationship between the energy I consume to heat my accommodation and the outdoor temperature, although other factors may also play a role.

How to *predict* how much energy I consume on average depending on the outdoor temperature ?

### Three Different Approaches to Home-Heating Modeling

| Process-based | Expert-based | *Statistical* |
| --- | --- | --- |
| Use some approximation of the heat equation in my accommodation given heat sources (radiators) and sinks (outdoor). | A thermal engineer diagnoses my accommodation based on his/her knowledge and/or on conventions. | Use energy-consumption and outdoor-temperature data to estimate parameters of a model. |
| <img alt="Building Energy Model" src="images/Heat_losses_of_the_building-fr.svg" width="180"> | <img alt="DPE" src="images/Diagnostic_de_performance_énergétique.svg" width="150"> | <img alt="Statistical Model" src="images/linear_ols.svg" width="280"> |

## Supervised Learning Objective

To define a supervised-learning problem we need:

- an *input* vector $\boldsymbol{X}$ of $X_1, \ldots, X_p$ input variables, or *features* and
- an *output* or *target* variable $Y$ used for supervision.

<hr>

**Supervised Learning Objective**
<br>
Construct the "best" prediction rule to predict $Y$ based on some *training data*: $(\boldsymbol{x}_i, y_i), i = 1, \ldots N$. 

<hr>

### Supervised-Learning Flow: Fit and Predict

<img alt="Pipeline Fit" src="images/api_diagram-predictor.fit.svg" width="500">

<img alt="Pipeline Fit" src="images/api_diagram-predictor.predict.svg" width="600">

### Supervised-Learning Flow: Transform-Fit and Transform-Predict

<img alt="Pipeline Fit" src="images/api_diagram-pipeline.fit.svg" width="750">

<img alt="Pipeline Fit" src="images/api_diagram-pipeline.predict.svg" width="860">

The $i$th *observation* of $X_j$ in the *sample* is given by the element $x_{ij}$ of the $N\times p$ *input-data matrix* $\mathbf{X}$.

The $i$th observation of $y$ is given by the element $y_i$ of the $N \times 1$ *output-data vector* $\mathbf{y}$.

> ***Question***
> - Recall the unbiased estimates of the sample mean $\bar{y}$ and the sample variance $s_Y^2$  of the random variable $Y$ for some data $y_i, 1 \le i \le N$.
> - Recall the unbiased estimate of the sample covariance $q_{X, Y}$ between $X$ and $Y$ for $(x_i, y_i), 1 \le i \le N$.

### Regression / Classification

| Regression | Classification |
| ------------------------------------- | --------------------------------------- |
| $Y$ is quantitative | $Y$ is qualitative |
| <img src="images/artur-solarz-hihmzc-TToc-unsplash.jpg" width="350"> | <img src="images/supervised.png" width="300"> |

### Example: Electricity Consumption Dependence on Temperature

- *Raw input*: temperature averaged over an administrative region of metropolitan France
- *Target*: regional electricity consumption

Let's first get familiar with the raw input data.

In [1]:
# Import modules
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Set data directory
data_dir = Path('data')

# Set keyword arguments for pd.read_csv
kwargs_read_csv = dict(header=0, index_col=0, parse_dates=True)

# Set first and last years
FIRST_YEAR = 2014
LAST_YEAR = 2021

# Define file path
filename = 'surface_temperature_merra2_{}-{}.csv'.format(
    FIRST_YEAR, LAST_YEAR)
filepath = Path(data_dir, filename)

# Read hourly temperature data averaged over each region
df_temp = pd.read_csv(filepath, **kwargs_read_csv).resample('D').mean()
temp_lim = [-5, 30]
label_temp = 'Temperature (°C)'

In [2]:
# Scatter plot of demand versus temperature
def plot_temp(region_name=df_temp.columns[0], year=FIRST_YEAR):
    df = df_temp[[region_name]].loc[str(year)]
    df.columns = [label_temp]
    nt = df.shape[0]
    std = float(df.std(0).iloc[0])
    mean = pd.Series(df[label_temp].mean(), index=df.index)
    df_std = pd.DataFrame(
        {'low': mean - std, 'high': mean + std}, index=df.index)
    cdf = pd.DataFrame(index=df.sort_values(by=label_temp).values[:, 0],
                       data=(np.arange(nt)[:, None] + 1) / nt)
    cdf.index.name = label_temp
    cdf.columns = ['Probability']

    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_figwidth(15)

    pts = df.plot(ax=axes[0],ylim=temp_lim, title='Time series, Mean, ± 1 STD')
    pts.hlines(df[label_temp].mean(), xmin=df.index[0], xmax=df.index[-1], linestyles='dashed')
    pts.fill_between(df_std.index.values, df_std.low, df_std.high, alpha=.2)

    pcdf = cdf.plot(ax=axes[1],xlim=temp_lim, ylim=[0, 1], title='Cumulative Distrib. Func.')
    pcdf.vlines(df[label_temp].mean(), ymin=0, ymax=1, linestyles='dashed')

    
    pkde = df.plot.kde(ax=axes[2],xlim=temp_lim, title='Probability Density Func.')
    ylim3 = pkde.get_ylim()
    pkde.vlines(df[label_temp].mean(), ymin=0, ymax=ylim3[1], linestyles='dashed')
    pkde.set_ylim([0, ylim3[1]])

In [3]:
# Show
from ipywidgets import interact
interact(plot_temp,region_name=df_temp.columns, year=range(FIRST_YEAR, LAST_YEAR))

interactive(children=(Dropdown(description='region_name', options=('Grand Est', 'Nouvelle-Aquitaine', 'Auvergn…

<function __main__.plot_temp(region_name='Grand Est', year=2014)>

> ***Question***
> - Describe how the mean and the standard deviation of the time series above depend on the region and on the year.
> - How do these dependencies show on the cumulative distribution and probability density functions?

Now let's compare the target data with the input data.

In [4]:
# Read hourly demand data summed over each region
filename = 'reseaux_energies_demand_demand.csv'
filepath = Path(data_dir, filename)
df_dem = pd.read_csv(filepath, **kwargs_read_csv).resample('D').sum()
label_dem = 'Demand (MWh)'

In [5]:
# Scatter plot of demand versus temperature
def scatter_temp_dem(region_name=df_temp.columns[0], year=FIRST_YEAR):
    df = pd.concat([df_temp[region_name], df_dem[region_name]],
                   axis='columns', ignore_index=True).loc[str(year)]
    df.columns = [label_temp, label_dem]
    df.plot.scatter(x=label_temp, y=label_dem, xlim=[-5, 30])

In [6]:
# Show
interact(scatter_temp_dem,region_name=df_temp.columns, year=range(FIRST_YEAR, LAST_YEAR))

interactive(children=(Dropdown(description='region_name', options=('Grand Est', 'Nouvelle-Aquitaine', 'Auvergn…

<function __main__.scatter_temp_dem(region_name='Grand Est', year=2014)>

## Supervised Learning Problem Definition

Given the output $Y$,

- define features $\boldsymbol{X} = (X_1, \ldots, X_p)$ based on (transformed) raw inputs
- define model by a function $f: \boldsymbol{X} \mapsto f(\boldsymbol{X})$
- define *loss function* $L(Y, f(\boldsymbol{X}))$
- choose a training set $(\boldsymbol{x}_i, y_i), i = 1, \ldots, N$

Example of model (linear): $f_{\boldsymbol{\beta}}(\boldsymbol{X}) = \beta_0 + \sum_{j = 1}^p X_j \beta_j$

Example of loss (squared error): $L(Y, f(\boldsymbol{X})) = \left(Y - f(\boldsymbol{X})\right)^2$

<div class="alert alert-block alert-warning">
    <b>Assumption</b>
    
All random variables and random vectors have finite variance and have densities (they are absolutely continuous with respect to the Lebesgue measure).
</div>

<hr>

**Expected Prediction Error**
<br>
For some model function $f$:

\begin{equation}
\mathrm{EPE}(f) = \mathbb{E}(L(Y, f(\boldsymbol{X})))
= \int L(y, f(\boldsymbol{x})) \rho_{\boldsymbol{X}, Y}(\boldsymbol{x}, y) d\boldsymbol{x} dy,
\end{equation}

where $\rho_{\boldsymbol{X}, Y}$ is the joint probability density of $\boldsymbol{X}$ and $Y$.
<hr>

<hr>

**Supervised Learning Objective (Concrete)**
<br>
Find $\hat{f}$ over all possible $f$ such that the EPE is minimized.

<hr>

#### Estimating the EPE

If the law of large numbers applies and for a fixed number $N$ of training data and an increasing number $N' - N$ of new (test) data,

\begin{equation}
    \frac{1}{N'} \sum_{i = 1}^{N'} L(y_i, f(\boldsymbol{x}_i)) \underset{N' \to \infty}{\to} \mathrm{EPE}(f).
\end{equation}

From the law of total expectation, we have that

\begin{align}
\mathrm{EPE}(f)
&= \mathbb{E}(\mathbb{E}[L(Y, f(X)) | X])\\
&= \int L(y, f(x)) \rho_{Y | X = x}(y) \rho_X(x)dy dx,
\end{align}

where $\rho_X$ is the probability density of $X$ and $\rho_{Y | X}$ is the conditional probability density of $Y$ knowing $X$.

The EPE can thus be interpreted as averaging over the inputs the prediction error for any input and can be minimized pointwise:

\begin{equation}
f(x) = \mathrm{argmin}_c \mathbb{E}\left[L(Y, c) | X = x\right].
\end{equation}

<div class="alert alert-block alert-warning">    
    <b>Remark</b>
    
    
Here, $f$ is the best possible model among all possible functions of $x$.

This is a theoretical notion.
    
In practice, we look for an $f$ in a set of models which is smaller than the set of all possible functions of $x$ (e.g. a linear model).
    
In that case, the model must be trained over all points in a relatively large sample so that the minimization cannot in general be done pointwise. 
</div>

## The Case of Squared Error Loss

The EPE using the squared error loss is

\begin{equation}
\mathrm{EPE}(f) = \mathbb{E}((Y - f(X))^2).
\end{equation}

Then
\begin{equation}
f(x) = \underset{c}{\mathrm{argmin}} \ \mathbb{E}\left[(Y - c)^2 | X = x\right].
\end{equation}

Since the expectation is the value that minimizes the expectation of the squared deviations (see [Appendix: Elements of Probability Theory](appendix_elements_of_probability_theory.ipynb)), the optimal solution is 

\begin{equation}
f(x) = \mathbb{E}(Y | X = x)
\end{equation}

In other words:

<div class="alert alert-block alert-info">
    <b>Theorem</b>
    
The best prediction of the output for any input is the conditional expectation, when best is measured by the average squared error and the optimum is looked for over all possible functions of $x$.
</div>

> ***Question (optional)***
> - What is the statistic giving the solution minimizing the EPE if we use the absolute error loss $|Y - f(X)|$ instead of the squared error loss?

## References

- [James, G., Witten, D., Hastie, T., Tibshirani, R., n.d. *An Introduction to Statistical Learning*, 2st ed. Springer, New York, NY.](https://www.statlearning.com/)
- Chap. 2, 3 and 7 in [Hastie, T., Tibshirani, R., Friedman, J., 2009. *The Elements of Statistical Learning*, 2nd ed. Springer, New York.](https://doi.org/10.1007/978-0-387-84858-7)
- Chap. 5 and 7 in [Wilks, D.S., 2019. *Statistical Methods in the Atmospheric Sciences*, 4th ed. Elsevier, Amsterdam.](https://doi.org/10.1016/C2017-0-03921-6)

***
## Credit

[//]: # "This notebook is part of [E4C Interdisciplinary Center - Education](https://gitlab.in2p3.fr/energy4climate/public/education)."
Contributors include Bruno Deremble and Alexis Tantet.
Several slides and images are taken from the very good [Scikit-learn course](https://inria.github.io/scikit-learn-mooc/).

<br>

<div style="display: flex; height: 70px">
    
<img alt="Logo LMD" src="images/logos/logo_lmd.jpg" style="display: inline-block"/>

<img alt="Logo IPSL" src="images/logos/logo_ipsl.png" style="display: inline-block"/>

<img alt="Logo E4C" src="images/logos/logo_e4c_final.png" style="display: inline-block"/>

<img alt="Logo EP" src="images/logos/logo_ep.png" style="display: inline-block"/>

<img alt="Logo SU" src="images/logos/logo_su.png" style="display: inline-block"/>

<img alt="Logo ENS" src="images/logos/logo_ens.jpg" style="display: inline-block"/>

<img alt="Logo CNRS" src="images/logos/logo_cnrs.png" style="display: inline-block"/>
    
</div>

<hr>

<div style="display: flex">
    <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0; margin-right: 10px" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a>
    <br>This work is licensed under a &nbsp; <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.
</div>